# L1_L2_Syndigo Mapping MachineLearning
  - module name: 2. L1_L2_Syndigo Mapping MachineLearning.ipynb
  - Purpose: Combine Level 1 & Level 2 from  Syndigo to build model

In [6]:
def prepare_data_1():
    pgm = inspect.currentframe().f_code.co_name  
    start_time = time.time() 
    # Reading PIMMART data
    pim_gtin_mapped = pd.read_csv(DBFR + "PIM_Data_New_50_82Mn.csv", dtype=object)
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD','PMY_DPT_CD', 'REC_DPT_CD', 'ITM_ID', 'GTIN']:
        pim_gtin_mapped[i] = pim_gtin_mapped[i].astype(np.float64)
    
    # Reading Syndigo 259K data
    synd_ALL = pd.read_csv(DBFR + 'Syndigo_Final_ALL.csv', dtype='unicode') # 259k Syndigo Data
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD', 'GTIN', 'ITM_ID', 'PMY_DPT_CD']:
        synd_ALL[i] = synd_ALL[i].astype(np.float64)
    
    # Stripping spaces from all columns
    df_obj = synd_ALL.select_dtypes(['object'])
    synd_ALL[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    
    syndigo_mapped = synd_ALL
    pimmart = pim_gtin_mapped
    syndigo_mapped.drop_duplicates('GTIN', inplace = True)
    syndigo_mapped['ITEM_SUBCOM_text'] = \
    (syndigo_mapped.VND_ECOM_DSC + ' ' + syndigo_mapped.SUBCOM_DSC).fillna('').str.lower()
    #syndigo_mapped['Level 1'].value_counts()
    #print(f'syndigo_mapped.shape {syndigo_mapped.shape}') 
    #print(f'syndigo_mapped.info() {syndigo_mapped.info()}') 
    #print(f'syndigo_mapped.head() {syndigo_mapped.head()}') 
    #print(f'syndigo_mapped.columns {syndigo_mapped.columns}') 
    
    syndigo_mapped['l1_l2'] = syndigo_mapped['Level 1'] + ' + ' + syndigo_mapped['Level 2']
    #print(f"syndigo_mapped['l1_l2'].value_counts() {syndigo_mapped['l1_l2'].value_counts(dropna = False)}")    
    return syndigo_mapped, pim_gtin_mapped

In [7]:
def prepare_data_2(samp_frac = 1):
    
    start_time = time.time()
    pgm = inspect.currentframe().f_code.co_name 
    global syndigo_mapped
    syndigo_mapped = syndigo_mapped_bkup.copy()
    print(f' samp_frac = {samp_frac}') 
    print (f'Before sample syndigo_mapped.shape {syndigo_mapped.shape }')
    
    vect = TfidfVectorizer(ngram_range = (1,2), max_features = 50000)
    whole_frac = 1
    if samp_frac  < whole_frac :   syndigo_mapped = syndigo_mapped.sample(frac = samp_frac,  random_state=42)
   
    print (f'After sample syndigo_mapped.shape {syndigo_mapped.shape }')
  
    print (f' syndigo_mapped .shape {syndigo_mapped .shape }')
    X = vect.fit_transform(syndigo_mapped.ITEM_SUBCOM_text) #scipy.sparse._csr.csr_matrix
    l1_l2_id_map = dict(zip(syndigo_mapped['l1_l2'].fillna('Other').unique(), range(syndigo_mapped['l1_l2'].fillna('Other').nunique())))
    id_l1_l2_map = dict(zip(range(syndigo_mapped['l1_l2'].fillna('Other').nunique()), syndigo_mapped['l1_l2'].fillna('Other').unique()))
    y  = syndigo_mapped['l1_l2'].fillna('Other').map(l1_l2_id_map)
    
    end_time = time.time()   
    desc = f' Elapse_time for "{pgm}".' 
    elapse_time (  start_time, end_time, desc)
    return  X, y, l1_l2_id_map, id_l1_l2_map

In [17]:
def grid_search(X, y):
    knn_grid = dict(n_neighbors=range(1, 21, 2),weights=['uniform', 'distance'],metric=['manhattan', 'minkowski'], p =[1, 2])
    lr_grid = dict(solver=['newton-cg', 'lbfgs', 'liblinear', 'saga', 'sag'],penalty=['l2'],C=[100, 10, 1.0, 0.1, 0.01], multi_class=['ovr','ovo', 'multinomial'])
    models = []
    models.append(('LR', LogisticRegression(), lr_grid))
    models.append(('KNN', KNeighborsClassifier(), knn_grid))
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    fit_mdl ={}
    for name, model, grid in models:    
        grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
        grid_result = grid_search.fit(X, y)
        fit_mdl[name] = grid_result
        # summarize results
        print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
        means = grid_result.cv_results_['mean_test_score']
        stds = grid_result.cv_results_['std_test_score']
        params = grid_result.cv_results_['params']
        for mean, stdev, param in zip(means, stds, params):
            print("%f (%f) with: %r" % (mean, stdev, param))
    return    

In [ ]:
grid_search(X, y)

In [3]:
def proc_predict(test_size):
     start_time = time.time()
     pgm = inspect.currentframe().f_code.co_name   
     global A_train, A_test, B_train, B_test, X_train, X_test, y_train, y_test
     A_train, A_test, B_train, B_test = train_test_split(syndigo_mapped.GTIN.tolist(), syndigo_mapped['l1_l2'].tolist(), test_size=test_size,  random_state=42)
     
     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test_size,  random_state=42)
     lr_tf = LogisticRegression(C = 100, multi_class = 'multinomial', solver = 'saga', n_jobs=-1)
     
     print(f"Training starts for test_size = {test_size} ")
     lr_tf.fit(X_train, y_train)
     preds = lr_tf.predict(X_test)
     #print(f"type(preds) {type(preds)} len(preds) {len(preds)} \n preds[0:10] {preds[0:10]}")
     preds_lrtf = preds
     probs = lr_tf.predict_proba(X_test)
     #print(f"type(probs) {type(probs)} len(probs) {len(probs)} \n probs[0:10] {probs[0:10]}")   
     preds_train = lr_tf.predict(X_train)
     #print(f"type(preds_train) {type(preds_train)} len(preds_train) {len(preds_train)} \n preds_train[0:10] {preds_train[0:10]} ")   
     probs_train = lr_tf.predict_proba(X_train)
     #print(f"type(probs_train) {type(probs_train)} len(probs_train) {len(probs_train)} \n probs_train[0:10] {probs_train[0:10]}")   
     #print(classification_report(y_test, preds_test,labels = lr_tf.classes_, target_names = [id2_level_map[i] for i in lr_tf.classes_]))
     
    
    
     #print("Accuracy = ",accuracy_score(y_test,preds))
     #print("Display TFIDF metrics")
     #print(classification_report(y_test, preds,labels = lr_tf.classes_, target_names = [id_l1_l2_map[i] for i in lr_tf.classes_]))
    
     print(f"Done Training ")
     end_time = time.time()   
     desc = f' Elapse_time for "{pgm}".' 
     elapse_time (  start_time, end_time, desc)
     return preds, preds_lrtf, probs, preds_train, probs_train, lr_tf 

In [4]:
def validation(sample_frec, test_size):
     start_time = time.time()
     pgm = inspect.currentframe().f_code.co_name     
     print(f'Validation for sample_frec = {sample_frec} and  test_size = {test_size}' ) 
     print("Accuracy = ",accuracy_score(y_test,preds))
     print (f' f1_score_weighted = {f1_score(y_test, preds, average="weighted")}')
     print (f" f1_score_macro = {f1_score(y_test, preds, average='macro')}")
     print (f" f1_score_micro = {f1_score(y_test, preds, average='micro')}")
     # Compute the Multiclass ROC AUC score
     #multi_class = 'multinomial'
     #score = roc_auc_score(y_test, preds, multi_class= multi_class)
     #print(f"ROC AUC score for Multiclass= {multi_class}: {score:.2f}")

     print("Display TFIDF metrics")
     test_metrics = pd.DataFrame(classification_report(y_test, preds,labels = lr_tf.classes_, target_names = [id_l1_l2_map[i] for i in lr_tf.classes_],  output_dict= True)).T
     #test_metrics = pd.DataFrame(classification_report(results_test.Actuals, results_test.Predictions, output_dict= True)).T 
     #print(test_metrics)
     #Stest_metrics.to_csv('L1_L2_Test_Metrics_yue.csv')
     tab_name = f'L1_L2_Test_Metrics_sample_frec{sample_frec}_test_size{test_size}'
     print(f'tab_name = {tab_name}')  
    
     end_time = time.time()   
     desc = f' Elapse_time for "{pgm}".' 
     elapse_time (  start_time, end_time, desc)    
     return test_metrics, tab_name 
                               

In [5]:
def proc_append():
    start_time = time.time()
    pgm = inspect.currentframe().f_code.co_name     
    test_new_proobs = []
    
    #print(f'len(preds)  {len(preds)}')
    for i in range(len(preds)):
        test_new_proobs.append(probs[i][np.argsort(probs[i])][::-1][:1].tolist())
    test_new_proobs = [element for sublist in list(test_new_proobs) for element in sublist]
    #print(f'test_new_proobs {test_new_proobs}')
    
    train_new_proobs = []
    
    #print(f'len(preds_train)  {len(preds_train)}')
    for i in range(len(preds_train)):
         test_new_proobs.append(probs_train[i][np.argsort(probs_train[i])][::-1][:1].tolist())
    train_new_proobs = [element for sublist in list(train_new_proobs) for element in sublist]
    #print(f'train_new_proobs {train_new_proobs}')  
    
    # process testLevels & trainLevelss 
    testLevels = []
    for j in y_test:
        testLevels.append([i for i in level2_id_map if level2_id_map[i]==j][0])
    #print(f'testLevels {testLevels [0:10]}')
    testLevelss = []
    for j in preds:
        testLevelss.append([i for i in level2_id_map if level2_id_map[i]==j][0])
    #print(f'testLevelss {testLevelss [0:10]}')
    
    trainLevelss = []
    for j in preds_train:
        trainLevelss.append([i for i in level2_id_map if level2_id_map[i]==j][0])
     
    end_time = time.time()   
    desc = f' Elapse_time for "{pgm}".' 
    elapse_time (  start_time, end_time, desc)    
    return  test_new_proobs, train_new_proobs, testLevels, testLevelss, trainLevelss

In [6]:
def proc_save():

    start_time = time.time()
    pgm = inspect.currentframe().f_code.co_name 
    print("---------------------------------------------\FINAL COUNTS:\n---------------------------------------------")
    print("1)", len(A_test + A_train))
    print("2)", len(['Test']*len(A_test) + ['Train']*len(A_train)))
    print("3)", len(B_test + B_train))
    print("4)", len(testLevels + B_train))
    print("5)", len(testLevelss + trainLevelss))
    print("6)", int(len(test_new_proobs + train_new_proobs)))
    
    data = {
        'GTIN' : A_test + A_train,
        'Source': ['Test']*len(A_test) + ['Train']*len(A_train),
        'Actual l1_l2' : B_test + B_train,
        'Actuals' : testLevels + B_train,
        'Predictions' : testLevelss + trainLevelss,
        'Scores' : test_new_proobs + train_new_proobs,
    
    }
    
    df = pd.DataFrame(data)
    df = df.merge(pim_gtin_mapped[['GTIN', 'ITM_ID', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD',
        'REC_DPT_DSC', 'DPT_CD', 'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD',
        'SUBCOM_DSC', 'VND_ECOM_DSC']], on='GTIN', how='left')
    df = df[['ITM_ID', 'GTIN', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD', 'REC_DPT_DSC', 'DPT_CD',
        'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD', 'SUBCOM_DSC',
        'VND_ECOM_DSC', 'Source', 'Actual l1_l2', 'Actuals', 'Predictions', 'Scores']]
    filename = 'Syndigo_Mapping_L1_L2_ML.csv'
    df.to_csv(DBFO + filename, index=False)
    print(f"{filename}  --file created successfully")

    end_time = time.time()   
    desc = f' Elapse_time for "{pgm}".' 
    elapse_time (  start_time, end_time, desc)
    return

In [1]:
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier,  RidgeClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score, f1_score
import re, time, inspect
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

pd.options.display.float_format = '{:20,.2f}'.format 
pd.set_option("display.max_rows", None)

# load  function of 'elapse_time'
path_code = 'C:\\users\\iny2819\\kroger\\Code\\'  
f_com_code = path_code + "com_code.py"
exec(compile(open(f_com_code , "rb").read(), f_com_code, 'exec' ))


In [2]:
DBFS = "/dbfs/FileStore/tables/OFFSHORE/"
DBFO = "/dbfs/FileStore/tables/OFFSHORE/"
DBFM = "/dbfs/FileStore/tables/MALLIK/"
DBFR = "/dbfs/FileStore/tables/OFFSHORE_RESULTS/"
path = 'C:\\users\\iny2819\\kroger\\Data\\'   
DBFS = path
DBFO = path
DBFM = path
DBFR = path


In [12]:
syndigo_mapped, pim_gtin_mapped = prepare_data_1()

In [13]:
#main
syndigo_mapped_bkup = syndigo_mapped.copy()

In [14]:
X, y, l1_l2_id_map, id_l1_l2_map = prepare_data_2(samp_frac = 0.3)

 samp_frac = 0.3
Before sample syndigo_mapped.shape (259085, 25)
After sample syndigo_mapped.shape (77726, 25)
 syndigo_mapped .shape (77726, 25)
  Elapse_time for "prepare_data_2". It took 2.367963 seconds - 0hh:0mm:2ss.
 start time: Aug 25 2023 13:22:08  end time:  Aug 25 2023 13:22:10


In [11]:
X, y, l1_l2_id_map, id_l1_l2_map = prepare_data_2(samp_frac = 0.3)
preds, preds_lrtf, probs, preds_train, probs_train, lr_tf = proc_predict(test_size = 0.3)
test_metric_1, tab_1 =  validation(sample_frec = 0.3, test_size = 0.3)

 samp_frac = 0.3
Before sample syndigo_mapped.shape (259085, 25)
After sample syndigo_mapped.shape (77726, 25)
 syndigo_mapped .shape (77726, 25)
  Elapse_time for "prepare_data_2". It took 2.861276 seconds - 0hh:0mm:2ss.
 start time: Aug 23 2023 01:31:01  end time:  Aug 23 2023 01:31:04
Training starts for test_size = 0.3 
Done Training 
  Elapse_time for "proc_predict". It took 111.158615 seconds - 0hh:1mm:51ss.
 start time: Aug 23 2023 01:31:04  end time:  Aug 23 2023 01:32:55
Validation for sample_frec = 0.3 and  test_size = 0.3
Accuracy =  0.8790633845098207
 f1_score_weighted = 0.8742430204178655
 f1_score_macro = 0.46950650637609476
 f1_score_micro = 0.8790633845098208
Display TFIDF metrics
tab_name = L1_L2_Test_Metrics_sample_frec0.3_test_size0.3
  Elapse_time for "validation". It took 0.095665 seconds - 0hh:0mm:0ss.
 start time: Aug 23 2023 01:32:55  end time:  Aug 23 2023 01:32:56


In [12]:

preds, preds_lrtf, probs, preds_train, probs_train, lr_tf = proc_predict(test_size = 0.1)
test_metric_2, tab_2 =  validation(sample_frec = 0.3, test_size = 0.1)

Training starts for test_size = 0.1 
Done Training 
  Elapse_time for "proc_predict". It took 154.818257 seconds - 0hh:2mm:34ss.
 start time: Aug 23 2023 01:32:56  end time:  Aug 23 2023 01:35:30
Validation for sample_frec = 0.3 and  test_size = 0.1
Accuracy =  0.8815129293708993
 f1_score_weighted = 0.8772497941434809
 f1_score_macro = 0.541360092896292
 f1_score_micro = 0.8815129293708993
Display TFIDF metrics
tab_name = L1_L2_Test_Metrics_sample_frec0.3_test_size0.1
  Elapse_time for "validation". It took 0.057141 seconds - 0hh:0mm:0ss.
 start time: Aug 23 2023 01:35:30  end time:  Aug 23 2023 01:35:30


In [14]:
syndigo_mapped = syndigo_mapped_bkup.copy()

In [16]:
X, y, l1_l2_id_map, id_l1_l2_map = prepare_data_2(samp_frac = 1)
preds, preds_lrtf, probs, preds_train, probs_train, lr_tf = proc_predict(test_size = 0.3)
test_metric_3, tab_3 = validation(sample_frec = 1, test_size = 0.3)

 samp_frac = 1
Before sample syndigo_mapped.shape (259085, 25)
After sample syndigo_mapped.shape (259085, 25)
 syndigo_mapped .shape (259085, 25)
  Elapse_time for "prepare_data_2". It took 9.952945 seconds - 0hh:0mm:9ss.
 start time: Aug 23 2023 01:50:22  end time:  Aug 23 2023 01:50:32
Training starts for test_size = 0.3 
Done Training 
  Elapse_time for "proc_predict". It took 494.629706 seconds - 0hh:8mm:14ss.
 start time: Aug 23 2023 01:50:32  end time:  Aug 23 2023 01:58:46
Validation for sample_frec = 1 and  test_size = 0.3
Accuracy =  0.8902426472480252
 f1_score_weighted = 0.8880122013849431
 f1_score_macro = 0.45908703124835426
 f1_score_micro = 0.8902426472480252
Display TFIDF metrics
tab_name = L1_L2_Test_Metrics_sample_frec1_test_size0.3
  Elapse_time for "validation". It took 0.371622 seconds - 0hh:0mm:0ss.
 start time: Aug 23 2023 01:58:46  end time:  Aug 23 2023 01:58:47


In [17]:
preds, preds_lrtf, probs, preds_train, probs_train, lr_tf = proc_predict(test_size = 0.1)
test_metric_4, tab_4 = validation(sample_frec = 1, test_size = 0.1)

Training starts for test_size = 0.1 
Done Training 
  Elapse_time for "proc_predict". It took 623.698597 seconds - 0hh:10mm:23ss.
 start time: Aug 23 2023 02:10:27  end time:  Aug 23 2023 02:20:51
Validation for sample_frec = 1 and  test_size = 0.1
Accuracy =  0.8931645374194296
 f1_score_weighted = 0.8907344356162481
 f1_score_macro = 0.5624715577166559
 f1_score_micro = 0.8931645374194297
Display TFIDF metrics
tab_name = L1_L2_Test_Metrics_sample_frec1_test_size0.1
  Elapse_time for "validation". It took 0.320256 seconds - 0hh:0mm:0ss.
 start time: Aug 23 2023 02:20:51  end time:  Aug 23 2023 02:20:52


In [21]:
#  Write classification report to excel
#  save() is going to remove, use close() instead
from pandas import ExcelWriter
from pandas import ExcelFile
excel_file = DBFO + 'L1_L2_Test_Metrics_cls_rpt.xlsx'

writer = pd.ExcelWriter(excel_file) 
tab_1 = 'L1_L2_TM_smp_frec0.3_tst_siz0.3'
tab_2 = 'L1_L2_TM_smp_frec0.3_tst_siz0.1'
tab_3 = 'L1_L2_TM_smp_frec1_tst_siz0.3'
tab_4 = 'L1_L2_TM_smp_frec1_tst_siz0.1'

test_metric_1.to_excel(writer,tab_1)
test_metric_2.to_excel(writer,tab_2)
test_metric_3.to_excel(writer,tab_3)
test_metric_4.to_excel(writer,tab_4)
writer.close()

C:\Users\iny2819\AppData\Local\Temp\ipykernel_24040\2099386414.py:16: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [ ]:
preds, preds_lrtf, probs, preds_train, probs_train, lr_tf = proc_predict(test_size = 0.1)

In [ ]:
test_metric_1, tab_1 =  validation(sample_frec = 0.3, test_size = 0.1)

In [ ]:
preds, preds_lrtf, probs, preds_train, probs_train, lr_tf = proc_predict(test_size = 0.3)

In [ ]:
test_metric_2, tab_2 =  validation(sample_frec = 0.3, test_size = 0.3)

In [ ]:
preds, preds_lrtf, probs, preds_train, probs_train, lr_tf = proc_predict(test_size = 0.3)

In [ ]:
#dic_test={"Hospitality Supplies":"hospitality_spplies", "Tobacco Products":"bacco_products"}
#dic_test={"Food / Beverages":"food_beverages"}


    X_subset, y_subset, subset_df, level2_id_map =  prepare_data_2(level__1)
    preds, preds_lrtf, probs, preds_train, probs_train = proc_predict(level__1)
    test_new_proobs, train_new_proobs, testLevels, testLevelss, trainLevelss = proc_append()
    proc_save(level__1, filenamee) 

## wip

In [ ]:
syndigo_mapped[['Level 1', 'Level 2']].value_counts()

In [ ]:
syndigo_mapped_bkup = syndigo_mapped.copy()

In [ ]:
syndigo_mapped['l1_l2'] = syndigo_mapped['Level 1'] + ' + ' + syndigo_mapped['Level 2']
syndigo_mapped.head()

In [ ]:
df_l1_l2 = pd.DataFrame(syndigo_mapped[['Level 1', 'Level 2']].value_counts(dropna = False).reset_index())
df_l1_l2 

In [ ]:
y[0:10]


In [ ]:
syndigo_mapped['l1_l2']

In [ ]:
syndigo_mapped['l1_l2'].value_counts(dropna =  False)

In [23]:
s = syndigo_mapped['l1_l2'].value_counts(dropna =  False)
s1 = s[s < 10]
s1

Marine + Heating, Ventilation, & Air Conditioning                                   9
Hardware + Safety & Security                                                        9
Electrical + Conduit, Raceways, Fittings & Accessories                              9
Tools + Benchtop & Stationary Tools                                                 8
Electronics + Optical Equipment                                                     8
Home & Venue Decoration + Wallpaper & Accessories                                   8
Sports & Outdoor Recreation Equipment + Basketball                                  8
Sports & Outdoor Recreation Equipment + Disc Golf, Ultimate Frisbee & Boomerangs    8
Office Supplies + Presentation Equipment                                            7
Furniture + Dining Room                                                             7
Electrical + Wall Plates                                                            7
Building Supplies + Rain Guttering                    

In [ ]:
data = {"count" : s1} 

df1 = pd.DataFrame(s, columns=["count"])   #NW Return enmpty DF
df1 = pd.DataFrame(data)
df1['count'].value_counts() 

In [25]:
y[0:10]

0    0
1    0
2    1
3    1
4    2
5    3
6    4
7    2
8    2
9    5
Name: l1_l2, dtype: int64

In [ ]:
getnnz(X)

In [ ]:
 test_metric_2, tab_2 =  validation(sample_frec = 0.3, test_size = 0.3)